# Week 10: 2016/03/28-04/01

## Monday

Happy Easter! No reading today since it's a holiday.

## Tuesday class

### Reducibility

Now that we've proven that one language ($A_{\mathsf{TM}}$) is undecidable, we can use it to prove that other languages are undecidable.

The first example is the halting problem. In many textbooks (and the poem "Scooping the Loop Snooper"), the halting problem is actually the prototypical undecidable language, but Sipser does things a little differently.

The proof is not difficult but it has a weird feel to it. The first thing that you have to remember is that the direction of the reduction is the opposite of what most people intuitively think of first. If you want to show that the halting problem is undecidable, you do _not_ reduce the halting problem to $A_{\mathsf{TM}}$. You assume that the halting problem _is_ decidable, then show via a reduction from $A_{\mathsf{TM}}$ to the halting problem that $A_{\mathsf{TM}}$ would also be decidable, which is a contradiction.

Theorems 5.2 and 3 are slightly different. They are both proofs of the undecidability of questions of the form "Does $L(M)$ belong to such-and-such a language class?". So the proofs go like this. Assume that the question is decidable, so there is a TM $C$ that decides it. Now we need to construct a new TM $S$ that decides $A_{\mathsf{TM}}$. Somehow, $S$ needs to use $C$, and $C$ expects to receive _another_ TM as input. So $S$ constructs another TM (called $M_1$ or $M_2$ in the book) for this purpose. As the book points out, the weird thing is that we do not actually run this machine; we just feed it into $C$.

The trick here is that the putative machine $C$ is able to do its job even if its argument doesn't halt on some inputs. The only hard part about deciding $A_{\mathsf{TM}}$ is detecting when a machine doesn't halt, so $S$ delegates this task to $C$.

If you like, you could think of $S$ as a Python function...

In [ ]:
# For Theorem 5.2
def accepts(function, input):
    def f(x):
        """Recognizes {w} if function(input) is True,
           recognizes \emptyset otherwise."""
        if x != input:
            return False
        else:
            return function(input)
    return not is_empty(f)

# For Theorem 5.3
def accepts(function, input):
    def f(x):
        """Recognizes \Sigma^\ast if function(input) is True,
           recognizes {0^n 1^n} otherwise."""
        n = len(x)/2
        if x == "0"*n+"1"*n:
            return True
        else:
            return function(input)
    return is_regular(f)

### Other undecidable problems

Alas, we don't have time to look at more examples of undecidable problems. The examples we've looked at so far may seem a little arcane because they are all questions about Turing machines, but some other examples are:

- Do two context-free grammars generate the same language (Exercise 5.1)?
- The third and hardest puzzle we did on the first day of class (Section 5.2).
- Does a polynomial equation in several variables have an integral solution?

See [the survey by Poonen](http://www-math.mit.edu/~poonen/papers/sampler.pdf) for more examples.

## Wednesday reading

Catch up by reading Section 5.1, up to but not including "Reductions via Computation Histories." 

If you were interested in the third, hardest puzzle that we did on the first day of class, then read the rest of Section 5.1 and Section 5.2, but this is optional.

## Thursday class

Today we'll discuss a question raised on the first day of class: am I a computer? Most people said no, but now we are armed with a formal definition of what a computer is. So the question becomes: Can my mind/brain be simulated by a Turing machine? This is a thorny question to say the least, but one that's worth at least discussing.

There aren't any notes for this topic save for one "no" argument that is technical. Turing called it "the mathematical objection" and it is usually known today as the Penrose-Lucas argument. This version is due to Penrose and comes from [an article criticizing him](http://www.ihmc.us/users/phayes/pub/lafortehayesford.pdf). 

Let $H$ be a Turing machine that detects _some_ cases of looping. That is, on input $\langle M, w\rangle$, if $H$ accepts, then $M$ loops on input $w$. But if $H$ rejects, it doesn't mean anything.

Now we go through the usual diagonalization argument; the only difference is that it doesn't lead to a contradiction, but to a machine/input pair that is beyond $H$'s detection abilities. Define $D$ to be the Turing machine that, on input $\langle M\rangle$, simulates $H$ on $\langle M, \langle M\rangle\rangle$. If $H$ accepts, then halt. If $H$ rejects, then go into an infinite loop. So,
\begin{equation*}
D(\langle D\rangle) \begin{cases}
\text{halts} & \text{if $H$ thinks $D$ loops on $\langle D\rangle$} \\
\text{loops} & \text{otherwise.}
\end{cases}
\end{equation*}
Since the first case is a contradiction, it must be that $H$ rejects $\langle D, \langle D\rangle\rangle$. So $D$ and $\langle D\rangle$ is an example of a machine/input pair that loops but $H$ is not able to detect.

Suppose that you are equivalent to a Turing machine. It should then be possible to construct a Turing machine $Y$ that accepts $\langle M, w\rangle$ in exactly those cases when you are able to show that $M$ loops on input $w$.

Then, by the above argument, there is a machine/input pair $\langle D, \langle D\rangle\rangle$ that loops, but $Y$ rejects it.  By the definition of $Y$, you are not able to detect that $\langle D, \langle D\rangle\rangle$. But if you understood the above argument, then you know that $\langle D, \langle D\rangle\rangle$ does loop. This is a contradiction!

There are many possible responses to this argument -- what do you think?